In [1]:
from zipline.api import order_target_percent, record, symbol, schedule_function, date_rules, order
from zipline.api import set_commission, commission, set_slippage, slippage
from zipline import run_algorithm
from zipline.utils.events import time_rules

import pandas as pd
import numpy as np

import requests
from datetime import datetime
from zipline.api import set_commission, set_slippage
from zipline.finance.commission import PerShare, PerDollar
from zipline.finance.slippage import VolumeShareSlippage

In [2]:
RETS_FILE_PATH = 'historical_returns.csv'

In [3]:
def get_weights_for_date(date):
  
    # The base URL of your Flask application
    base_url = "http://localhost:5000/get_weights"
    
    # Prepare the query parameters
    params = {'date': str(date)}
    
    try:
        # Sending a GET request to the endpoint
        response = requests.get(base_url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            return data.get('weights')
        else:
            # Handle possible errors (e.g., invalid date format, missing date parameter)
            print(f"Error: {response.json()['error']}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [4]:
# Define the initialize function
def initialize(context):

        # Set the commission model
    #set_commission(PerDollar(cost=0.0001))
    set_commission(PerDollar(cost=0))
    
    # Set the slippage model
    # VolumeShareSlippage takes 'volume_limit' and 'price_impact' as parameters
    # 'volume_limit' is the fraction of a bar's volume that your order can take up
    # 'price_impact' is the impact your order has on the price per share for each share traded
    #set_slippage(VolumeShareSlippage(volume_limit=0.025, price_impact=0.01))
    
    # Slippage model: Assuming 0.05% slippage per trade
    #set_slippage(slippage.VolumeShareSlippage(volume_limit=0.025, price_impact=0.0005))

    
    context.tickers = ['VWO', 'SPY', 'VNQ', 'LQD', 'DBC']
    context.assets = [symbol(ticker) for ticker in context.tickers]
    
    # Schedule the rebalance function to run every Monday at market open
    schedule_function(rebalance, date_rules.week_start(), time_rules.market_open())
    # Set the context attributes needed
    context.lookback_days = 50 # look back over 50 days for the moving average

# Define the rebalance function
def rebalance(context, data):
    #print(context.get_datetime().date())
     
    # Get historical data for the assets
    historical_data = data.history(context.assets, "price", context.lookback_days + 1, "1d")
    
#     # Calculate daily returns
    daily_returns = historical_data.pct_change().dropna()
    daily_returns.to_csv(RETS_FILE_PATH)
    current_date = context.get_datetime().date()
    
    print(f'Handling {current_date}')
    
    weights = get_weights_for_date(current_date)
    print(weights)
    
    # Place orders
    for asset, weight in zip(context.assets, weights):
        if data.can_trade(asset):
            order_target_percent(asset, weight)
            #print(f'ordered {asset}: {weight}')
    
    # Record some metrics
    record(weights=weights)

# Define the analyze function (optional)
def analyze(context, perf):
    # Plot portfolio weights over time
    #perf.weights.plot()
    pass

In [5]:
old_dates  = [
    (pd.Timestamp('2017-12-15', tz='UTC'), pd.Timestamp('2018-12-15', tz='UTC')),
    (pd.Timestamp('2018-12-15', tz='UTC'), pd.Timestamp('2019-12-15', tz='UTC')),
    (pd.Timestamp('2019-12-15', tz='UTC'), pd.Timestamp('2020-12-15', tz='UTC')),
    (pd.Timestamp('2017-12-15', tz='UTC'), pd.Timestamp('2020-12-15', tz='UTC')),
] 


new_dates  = [
    (pd.Timestamp('2020-12-15', tz='UTC'), pd.Timestamp('2021-12-15', tz='UTC')),
    (pd.Timestamp('2021-12-15', tz='UTC'), pd.Timestamp('2022-12-15', tz='UTC')),
    (pd.Timestamp('2022-12-15', tz='UTC'), pd.Timestamp('2023-12-15', tz='UTC')),
    (pd.Timestamp('2020-12-15', tz='UTC'), pd.Timestamp('2023-12-15', tz='UTC')),
] 

In [48]:
dates = old_dates
# dates = new_dates

In [51]:
perfs = []
capital_base = 1e7

for ds in dates:
    start_date, end_date = ds
    print(f'\n*********\n   ({start_date.date()}, {end_date.date()}) \n*********\n')
    perf = run_algorithm(
        start=start_date,
        end=end_date,
        initialize=initialize,
        analyze=analyze,
        capital_base=capital_base,
        data_frequency='daily',
        bundle='thesis1' 
    )
    perfs.append(perf)


*********
   (2017-12-15, 2018-12-15) 
*********

Handling 2017-12-18
[-0.07895562052726746, 0.610747754573822, -0.42514437437057495, 0.352143257856369, 0.5412089824676514]
Handling 2017-12-26


C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(


[0.1999300867319107, 0.029027244076132774, -0.446811318397522, 0.6569528579711914, 0.5609009265899658]
Handling 2018-01-02
[-0.2609204649925232, 1.000011682510376, 0.018998660147190094, 0.25231871008872986, -0.010408598929643631]
Handling 2018-01-08
[-0.37569138407707214, 0.9221847653388977, -0.3145352303981781, 0.5633494853973389, 0.20469236373901367]
Handling 2018-01-16
[0.08286765217781067, 0.6019934415817261, -0.06511683017015457, 0.4842383563518524, -0.10398279130458832]
Handling 2018-01-22
[-0.5916204452514648, 0.09500295668840408, -0.2590821385383606, 0.8835954666137695, 0.8721041083335876]
Handling 2018-01-29
[-0.2745298743247986, 0.30626749992370605, 0.3166545331478119, 0.2936939299106598, 0.35791370272636414]
Handling 2018-02-05
[0.04099847376346588, 0.33555465936660767, 0.2088766247034073, 0.35037505626678467, 0.06419496983289719]
Handling 2018-02-12
[-0.0016976884799078107, 0.4080347716808319, 0.2941553294658661, 0.11434678733348846, 0.18516042828559875]
Handling 2018-02-20

[-0.08653286844491959, 0.02429950423538685, -0.0013498933985829353, 0.8801557421684265, 0.18342742323875427]
Handling 2019-04-08
[0.112483911216259, 0.5769523978233337, -0.031647689640522, 0.3288235664367676, 0.013387700542807579]
Handling 2019-04-15
[0.02540529891848564, 0.47123128175735474, -0.26570430397987366, 0.6069693565368652, 0.16209831833839417]
Handling 2019-04-22
[-0.2305348962545395, 0.6675070524215698, -0.1705167144536972, 0.8382554650306702, -0.10471096634864807]
Handling 2019-04-29
[-0.0980035588145256, 0.6868864297866821, 0.0782754197716713, 0.39026179909706116, -0.05742025375366211]
Handling 2019-05-06
[-0.04307498782873154, 0.4290899634361267, -0.14999271929264069, 0.5675786137580872, 0.19639913737773895]
Handling 2019-05-13
[0.09630607813596725, 0.41840723156929016, -0.1316668838262558, 0.6411892175674438, -0.024235744029283524]
Handling 2019-05-20
[-0.27691012620925903, 0.2923930883407593, -0.07369308918714523, 0.05820225924253464, 1.0000076293945312]
Handling 2019-

[0.0866360068321228, 0.268777072429657, 0.04240075871348381, 0.5426599383354187, 0.059526123106479645]
Handling 2020-07-06
[0.06488785147666931, 0.41108936071395874, -0.18424509465694427, 0.7060977220535278, 0.0021701210644096136]
Handling 2020-07-13
[-0.4124926030635834, 0.5477932691574097, 0.060410063713788986, 0.47291046380996704, 0.3313787877559662]
Handling 2020-07-20
[-0.3170217275619507, 0.19513286650180817, 0.24696022272109985, 0.427135705947876, 0.44779279828071594]
Handling 2020-07-27
[-0.49186480045318604, 0.5566923022270203, 0.07277035713195801, 0.662240207195282, 0.20016179978847504]
Handling 2020-08-03
[-0.22596849501132965, 0.8512712121009827, 0.1014634221792221, 0.44219815731048584, -0.16896437108516693]
Handling 2020-08-10
[-0.4352322518825531, 0.43501392006874084, -0.0002910156035795808, 0.8487988710403442, 0.15171046555042267]
Handling 2020-08-17
[0.0356764979660511, 0.3162075877189636, 0.14400772750377655, 0.47173982858657837, 0.032368287444114685]
Handling 2020-08-

C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(


[0.1999300867319107, 0.029027244076132774, -0.446811318397522, 0.6569528579711914, 0.5609009265899658]
Handling 2018-01-02
[-0.2609204649925232, 1.000011682510376, 0.018998660147190094, 0.25231871008872986, -0.010408598929643631]
Handling 2018-01-08
[-0.37569138407707214, 0.9221847653388977, -0.3145352303981781, 0.5633494853973389, 0.20469236373901367]
Handling 2018-01-16
[0.08286765217781067, 0.6019934415817261, -0.06511683017015457, 0.4842383563518524, -0.10398279130458832]
Handling 2018-01-22
[-0.5916204452514648, 0.09500295668840408, -0.2590821385383606, 0.8835954666137695, 0.8721041083335876]
Handling 2018-01-29
[-0.2745298743247986, 0.30626749992370605, 0.3166545331478119, 0.2936939299106598, 0.35791370272636414]
Handling 2018-02-05
[0.04099847376346588, 0.33555465936660767, 0.2088766247034073, 0.35037505626678467, 0.06419496983289719]
Handling 2018-02-12
[-0.0016976884799078107, 0.4080347716808319, 0.2941553294658661, 0.11434678733348846, 0.18516042828559875]
Handling 2018-02-20

[-0.08653286844491959, 0.02429950423538685, -0.0013498933985829353, 0.8801557421684265, 0.18342742323875427]
Handling 2019-04-08
[0.112483911216259, 0.5769523978233337, -0.031647689640522, 0.3288235664367676, 0.013387700542807579]
Handling 2019-04-15
[0.02540529891848564, 0.47123128175735474, -0.26570430397987366, 0.6069693565368652, 0.16209831833839417]
Handling 2019-04-22
[-0.2305348962545395, 0.6675070524215698, -0.1705167144536972, 0.8382554650306702, -0.10471096634864807]
Handling 2019-04-29
[-0.0980035588145256, 0.6868864297866821, 0.0782754197716713, 0.39026179909706116, -0.05742025375366211]
Handling 2019-05-06
[-0.04307498782873154, 0.4290899634361267, -0.14999271929264069, 0.5675786137580872, 0.19639913737773895]
Handling 2019-05-13
[0.09630607813596725, 0.41840723156929016, -0.1316668838262558, 0.6411892175674438, -0.024235744029283524]
Handling 2019-05-20
[-0.27691012620925903, 0.2923930883407593, -0.07369308918714523, 0.05820225924253464, 1.0000076293945312]
Handling 2019-

[0.06488785147666931, 0.41108936071395874, -0.18424509465694427, 0.7060977220535278, 0.0021701210644096136]
Handling 2020-07-13
[-0.4124926030635834, 0.5477932691574097, 0.060410063713788986, 0.47291046380996704, 0.3313787877559662]
Handling 2020-07-20
[-0.3170217275619507, 0.19513286650180817, 0.24696022272109985, 0.427135705947876, 0.44779279828071594]
Handling 2020-07-27
[-0.49186480045318604, 0.5566923022270203, 0.07277035713195801, 0.662240207195282, 0.20016179978847504]
Handling 2020-08-03
[-0.22596849501132965, 0.8512712121009827, 0.1014634221792221, 0.44219815731048584, -0.16896437108516693]
Handling 2020-08-10
[-0.4352322518825531, 0.43501392006874084, -0.0002910156035795808, 0.8487988710403442, 0.15171046555042267]
Handling 2020-08-17
[0.0356764979660511, 0.3162075877189636, 0.14400772750377655, 0.47173982858657837, 0.032368287444114685]
Handling 2020-08-24
[-0.05784240737557411, 0.26837390661239624, 0.11223728209733963, 0.5549089312553406, 0.12232215702533722]
Handling 2020-

In [52]:
## Sharpe Ratio: 5.319768807195642

import empyrical as ep

sharpes = []

for perf in perfs:
#     cumulative_returns = ep.cum_returns_final(perf['returns'])
#     annualized_returns = ep.annual_return(perf['returns'])
    sharpe_ratio = ep.sharpe_ratio(perf['returns'])
    #max_drawdown = ep.max_drawdown(perf['returns'])
    sharpes.append(sharpe_ratio)

In [53]:
sharpes

[4.0277695212412725,
 0.23065909356679515,
 0.8356575175187253,
 1.4022050696412431]

## Save the result

In [12]:
len(perfs)

4

In [47]:
PERF_NAME = r'new\rnn_min_var'
perfs[3].to_csv(fr'..\analytics\perfs\{PERF_NAME}_perf.csv')

# Analytics

In [ ]:
# mean_returns: -0.00022047444443614898
# std: 0.006793907584478022
# Cumulative Returns: -0.1673630146755024
# Annualized Returns: -0.05907459961792738
# Sharpe Ratio: -0.5131137893372564
# Max Drawdown: -0.2874978151290583

In [42]:
## Sharpe Ratio: 5.319768807195642

import empyrical as ep

sharpes = []

for perf in perfs:
#     cumulative_returns = ep.cum_returns_final(perf['returns'])
#     annualized_returns = ep.annual_return(perf['returns'])
    sharpe_ratio = ep.sharpe_ratio(perf['returns'])
    #max_drawdown = ep.max_drawdown(perf['returns'])
    sharpes.append(sharpe_ratio)

In [ ]:
tmp = [perf['weights'][i][1] for i in range(4, len(perf['weights']))]
tmp = np.array(tmp)
tmp[tmp > 1]

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

perf['cumulative_returns'] = (1 + perf['returns']).cumprod() - 1

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(perf['cumulative_returns'], label='Cumulative Returns')
plt.title('Cumulative Returns Over Time')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()

In [ ]:
perf['weights']

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

# Define the dates
start_date = datetime(2007, 1, 3)
retraining_dates = [datetime(2021, 12, 15), datetime(2022, 12, 15)]
test_start_date = datetime(2021, 12, 15)
end_date = datetime(2022, 12, 15)

# Plotting
fig, ax = plt.subplots(figsize=(10, 2))

# Highlight the training period
ax.plot([start_date, test_start_date], [1, 1], color="skyblue", linewidth=8, label='Training Period')

# Highlight the testing period
ax.plot([test_start_date, end_date], [1, 1], color="lightgreen", linewidth=8, label='Testing Period')

# Mark the retraining dates
for date in retraining_dates:
    plt.plot([date, date], [0.8, 1.2], color="red", linestyle='--', linewidth=2)

# Adding annotations for retraining dates
for date in retraining_dates:
    ax.text(date, 1.25, '', ha='center', color='red')

# Adjust the plot
ax.set_ylim(0.75, 1.5)
ax.yaxis.set_visible(False)  # Hide the y-axis
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Adding labels and title
plt.title('Timeline of Training, Testing, and Model Retraining')

# Adding custom legend
handles, labels = ax.get_legend_handles_labels()
custom_lines = [plt.Line2D([0], [0], color="skyblue", lw=8),
                plt.Line2D([0], [0], color="lightgreen", lw=8),
                plt.Line2D([0], [0], color="red", linestyle='--', lw=2)]
plt.legend(custom_lines, ['Training Period', 'Testing Period', 'Retraining'])

plt.tight_layout()
plt.show()
